# Кидаем камень

## Подготовка

Добавим все библиотеки, которые мы будем использовать в окружение.

In [16]:
from ipycanvas import Canvas # Канва для отображения действий
import ipywidgets as wg # Виджеты для интерактива

import typing # Строгая типизация

import math # Математические формулы

from vector import Vector # Самописная малюсенькая библиотека для работы с векторами

Для начала создаем канву и метод, с помощью которого мы будем рисовать положения тела в пространстве после его броска нами.

В данном случае будем использовать черный прямоугольный камень. Сопротивлением воздуха пренебречь

In [17]:
width = 1024
height = 256

# Канва
canvas = Canvas(width = width, height = height)

stone_color = "black"
angle_line_color = "blue"

Создадим так же несколько методов для рисования нужных нам примитивов

In [18]:
def clear_canvas():
    """Очистка канвы"""
    global canvas

    canvas = Canvas(width=width, height=height)
    canvas.clear()


def connect_two_point(start: Vector, end: Vector, color: str):
    """Линия между двумя точками"""
    global canvas

    canvas.move_to(start.x, start.y)
    canvas.line_to(end.x, end.y)
    canvas.stroke_style = color
    canvas.stroke()

def fill_rect(position: Vector, size: Vector, color: str):
    global canvas
    
    canvas.fill_style = color
    canvas.fill_rect(position.x, position.y, size.x, size.y)


def draw_point(position: Vector):
    """Рисуем камень"""
    size = Vector(10, 10)
    fill_rect(position, size, stone_color)
   

def draw_line(angle):
    """Отрисовка линии угла броска"""
    start = Vector(0, height)
    rad_angle = math.radians(angle)
    end = Vector(
        math.cos(rad_angle) * min(width, height), # Линия видна по всей канве 
        height - math.sin(rad_angle) * height # Система отсчета канвы - от верхнего левого угла, удобнее - од нижнего левого => переворачиваеем (height - y)
    )
    clear_canvas()
    connect_two_point(start, end, angle_line_color)

## Математическая составляющая

Мы будем считать положение тела по формуле $\vec{P}(t) = \vec{P_0} + \vec{V_0}t + \frac{\vec{a}t^2}{2}$, где: 
- $P$ - положение тела в текущий момент
- $P_0$ - начальное положение тела
- $V_0$ - начальная скорость
- $a$ - ускорение тела
- $t$ - время прошедшее с момента броска

In [19]:
def next_position(start_position: Vector, start_velocity: Vector, acceleration: Vector):
    """Функция с замыканием для рассчета следующего положения в зависимости от аргументов и времени"""
    def future_position(t: float) -> Vector:
        return start_position \
            + (start_velocity * t) \
            + (acceleration * (t ** 2.0)) / 2.0 

    return future_position

### Задание начальных условий

Для начала положим, что мы начинаем движение из точки $P_0(0;0)$ со скоростью ```k_velocity``` метров в секунду и углом ```start_angle``` радиан

In [20]:
g = Vector(0, -9.81) # Ускорение свободного падения (направлено вниз)

position = Vector(0, 0)

full_time = 60 # Cколько будем просчитывать (в секундах)

def show_line(angle, delta_time, force):
    draw_line(angle)

    start_velocity = Vector.from_angle(math.radians(angle)) * force

    func = next_position(position, start_velocity, g)

    slices = [func(float(i) * delta_time) for i in range(0, int(full_time / delta_time))]

    for i in slices:
        draw_point(Vector(i.x, height - i.y))

    return canvas

start_angle_w = wg.FloatSlider(description="Угол броска", min=0, max=90, step=0.5)
delta_time_w = wg.FloatSlider(description="Дельта времени", min=0.1, max=4, step=0.5) # Через какой отрезок времени будем делать срезы объекта
force_w = wg.FloatSlider(description="Сила, с которой кидаем камень", min=0.0, max=100.0, step = 0.5)

_ = wg.interact(show_line, angle=start_angle_w, delta_time=delta_time_w, force=force_w)

interactive(children=(FloatSlider(value=0.0, description='Угол броска', max=90.0, step=0.5), FloatSlider(value…